In [2]:
import requests

In [3]:
from openai import OpenAI

In [4]:
import json

In [5]:
from dotenv import load_dotenv

In [9]:
load_dotenv()

True

In [10]:
import os

In [11]:
stock_api = os.getenv("ALPHA_VANTAGE_API_KEY")

In [12]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [ ]:
url = "https://www.alphavantage.co/query?"

In [55]:
def find_ticker(text):
    """Based on the natural language query, use this function to find the ticker symbol"""

    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": text,
        "apikey": stock_api
    }

    response = requests.get(url, params)

    data = response.json()

    ticker = (data.get('bestMatches')[0]).get('1. symbol')

    return ticker

In [56]:
find_ticker('Tesla')

'TSLA'

In [57]:
def get_income_statement(ticker):
    """Based on the ticker found out using the function find_ticker(text) you can find the income statement of the company from this
    function. This will fetch both quarterly and annual income statements.
    """

    params = {
        "function": "INCOME_STATEMENT",
        "symbol": ticker,
        "apikey": stock_api
    }

    income_statement = requests.get(url,params)

    income_statement_data = income_statement.json()

    return income_statement_data

In [116]:
ticker_symbol = find_ticker('tesla')

In [64]:
cached_income_statement = get_income_statement(a)

cached_income_statement

{'symbol': 'TSLA',
 'annualReports': [{'fiscalDateEnding': '2024-12-31',
   'reportedCurrency': 'USD',
   'grossProfit': '17450000000',
   'totalRevenue': '97690000000',
   'costOfRevenue': '80240000000',
   'costofGoodsAndServicesSold': '80240000000',
   'operatingIncome': '7076000000',
   'sellingGeneralAndAdministrative': '5150000000',
   'researchAndDevelopment': '4540000000',
   'operatingExpenses': '10374000000',
   'investmentIncomeNet': 'None',
   'netInterestIncome': '1219000000',
   'interestIncome': '1569000000',
   'interestExpense': '350000000',
   'nonInterestIncome': 'None',
   'otherNonOperatingIncome': 'None',
   'depreciation': 'None',
   'depreciationAndAmortization': '5368000000',
   'incomeBeforeTax': '8990000000',
   'incomeTaxExpense': '1837000000',
   'interestAndDebtExpense': 'None',
   'netIncomeFromContinuingOperations': '7153000000',
   'comprehensiveIncomeNetOfTax': 'None',
   'ebit': '9340000000',
   'ebitda': '14708000000',
   'netIncome': '7130000000'},


In [122]:
def get_annual_income_statement(json_file):

    """ 
    You are supposed to use this function, if the query is related to finding annual income statement.
    """

    annual_filter = json_file.get('annualReports')

    filtered_data = []

    for report in annual_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'grossProfit': report.get('grossProfit'),
            'totalRevenue': report.get('totalRevenue'),
            'costOfRevenue': report.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
            'operatingIncome': report.get('operatingIncome'),
            'operatingExpenses': report.get('operatingExpenses'),
            'researchAndDevelopment': report.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report.get('netInterestIncome'),
            'interestIncome': report.get('interestIncome'),
            'depreciationAndAmortization': report.get('depreciationAndAmortization'),
            'incomeBeforeTax': report.get('incomeBeforeTax'),
            'incomeTaxExpense': report.get('incomeTaxExpense'),
            'ebit': report.get('ebit'),
            'ebitda': report.get('ebitda'),
            'netIncome': report.get('netIncome'),
        })

    return filtered_data

In [101]:
get_annual_income_statement(cached_income_statement)

[{'fiscalDateEnding': '2024-12-31',
  'grossProfit': '17450000000',
  'totalRevenue': '97690000000',
  'costOfRevenue': '80240000000',
  'costofGoodsAndServicesSold': '80240000000',
  'operatingIncome': '10374000000',
  'researchAndDevelopment': '4540000000',
  'sellingGeneralAndAdministrative': '5150000000',
  'netInterestIncome': '1219000000',
  'interestIncome': '1569000000',
  'depreciationAndAmortization': '5368000000',
  'incomeBeforeTax': '8990000000',
  'incomeTaxExpense': '1837000000',
  'ebit': '9340000000',
  'ebitda': '14708000000',
  'netIncome': '7130000000'},
 {'fiscalDateEnding': '2023-12-31',
  'grossProfit': '17660000000',
  'totalRevenue': '96773000000',
  'costOfRevenue': '79113000000',
  'costofGoodsAndServicesSold': '79113000000',
  'operatingIncome': '8769000000',
  'researchAndDevelopment': '3969000000',
  'sellingGeneralAndAdministrative': '4800000000',
  'netInterestIncome': '910000000',
  'interestIncome': '1066000000',
  'depreciationAndAmortization': '46670

In [121]:
def get_quarterly_income_statement(json_file):

    """ 
    You are supposed to use this function, if the query is related to finding annual income statement datapoints.
    """

    quarterly_filter = json_file.get('quarterlyReports')

    filtered_qoq_data = []

    for report in quarterly_filter:
        filtered_qoq_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'grossProfit': report.get('grossProfit'),
            'totalRevenue': report.get('totalRevenue'),
            'costOfRevenue': report.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
            'operatingIncome': report.get('operatingIncome'),
            'operatingExpenses': report.get('operatingExpenses'),
            'researchAndDevelopment': report.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report.get('netInterestIncome'),
            'interestIncome': report.get('interestIncome'),
            'depreciationAndAmortization': report.get('depreciationAndAmortization'),
            'incomeBeforeTax': report.get('incomeBeforeTax'),
            'incomeTaxExpense': report.get('incomeTaxExpense'),
            'ebit': report.get('ebit'),
            'ebitda': report.get('ebitda'),
            'netIncome': report.get('netIncome'),
        })


    return filtered_qoq_data

In [109]:
get_quarterly_income_statement(cached_income_statement)

[{'fiscalDateEnding': '2025-09-30',
  'grossProfit': '5054000000',
  'totalRevenue': '28095000000',
  'costOfRevenue': '23041000000',
  'costofGoodsAndServicesSold': '23041000000',
  'operatingIncome': '3430000000',
  'researchAndDevelopment': '1630000000',
  'sellingGeneralAndAdministrative': '1562000000',
  'netInterestIncome': '363000000',
  'interestIncome': '439000000',
  'depreciationAndAmortization': '1625000000',
  'incomeBeforeTax': '1959000000',
  'incomeTaxExpense': '570000000',
  'ebit': '2035000000',
  'ebitda': '3660000000',
  'netIncome': '1373000000'},
 {'fiscalDateEnding': '2025-06-30',
  'grossProfit': '3878000000',
  'totalRevenue': '22496000000',
  'costOfRevenue': '18618000000',
  'costofGoodsAndServicesSold': '18618000000',
  'operatingIncome': '2955000000',
  'researchAndDevelopment': '1589000000',
  'sellingGeneralAndAdministrative': '1366000000',
  'netInterestIncome': '306000000',
  'interestIncome': '392000000',
  'depreciationAndAmortization': '1433000000',


In [112]:
def get_balance_sheet(ticker):
    """Based on the ticker found out using the function find_ticker(text) you can find the balance sheet of the company from this
    function. This will fetch both quarterly and annual income statements.
    """

    params = {
        "function": "BALANCE_SHEET",
        "symbol": ticker,
        "apikey": stock_api
    }

    balance_sheet = requests.get(url,params)

    balance_sheet_data = balance_sheet.json()

    return balance_sheet_data

In [119]:
cached_balance_sheet = get_balance_sheet(ticker_symbol)

cached_balance_sheet

{'symbol': 'TSLA',
 'annualReports': [{'fiscalDateEnding': '2024-12-31',
   'reportedCurrency': 'USD',
   'totalAssets': '122070000000',
   'totalCurrentAssets': '58360000000',
   'cashAndCashEquivalentsAtCarryingValue': '16139000000',
   'cashAndShortTermInvestments': '16139000000',
   'inventory': '12017000000',
   'currentNetReceivables': '4418000000',
   'totalNonCurrentAssets': '63716000000',
   'propertyPlantEquipment': '51501000000',
   'accumulatedDepreciationAmortizationPPE': 'None',
   'intangibleAssets': '1226000000',
   'intangibleAssetsExcludingGoodwill': '1226000000',
   'goodwill': '244000000',
   'investments': 'None',
   'longTermInvestments': 'None',
   'shortTermInvestments': '20424000000',
   'otherCurrentAssets': '5362000000',
   'otherNonCurrentAssets': 'None',
   'totalLiabilities': '48390000000',
   'totalCurrentLiabilities': '28821000000',
   'currentAccountsPayable': '12474000000',
   'deferredRevenue': 'None',
   'currentDebt': 'None',
   'shortTermDebt': '32

In [123]:
def annual_balance_sheet(json_data):

    """ 
    You are supposed to use this function, if the query is related to finding annual balance sheet datapoints.
    """

    annual_filter = json_data.get('annualReports')

    filtered_data = []

    for report in annual_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'totalAssets': report.get('totalAssets'),
            'totalCurrentAssets': report.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report.get('inventory'),
            'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report.get('totalLiabilities'),
            'interestIncome': report.get('interestIncome'),
            'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report.get('currentLongTermDebt'),
            'totalShareholderEquity': report.get('totalShareholderEquity'),
            'retainedEarnings': report.get('retainedEarnings'),
            'commonStock': report.get('commonStock')
        })

    return filtered_data

In [125]:
def quarterly_balance_sheet(json_data):

    """ 
    You are supposed to use this function, if the query is related to finding quarterly balance sheet datapoints.
    """

    quarterly_filter = json_data.get('quarterlyReports')

    filtered_data = []

    for report in quarterly_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'totalAssets': report.get('totalAssets'),
            'totalCurrentAssets': report.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report.get('inventory'),
            'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report.get('totalLiabilities'),
            'interestIncome': report.get('interestIncome'),
            'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report.get('currentLongTermDebt'),
            'totalShareholderEquity': report.get('totalShareholderEquity'),
            'retainedEarnings': report.get('retainedEarnings'),
            'commonStock': report.get('commonStock')
        })

    return filtered_data